# Case

In [1]:
import numpy as np
import pandas as pd

from typing import Dict

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
!ams misc --version

Python   3.12.0
ams      1.0.8
andes    1.9.3.post6+g518882a0
numpy    2.0.2
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [4]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=20)

In [5]:
mpc5 = ams.io.matpower.m2mpc("./Texas2k/Texas2k_case5.m")

 Case saved by PowerWorld Simulator, version 23, build date October 25, 2023


Tune the parameters

In [6]:
gencost = np.concatenate((mpc5['gencost'][:, 0:4],
                          np.zeros((mpc5["gencost"].shape[0], 4))),
                         axis=1)

gencost[:, 3] = 2  # type 2 gencost
gencost[:, 5] = 0.0  # c2
gencost[:, 6] = mpc5['gencost'][:, 5]  # c1
gencost[:, 7] = mpc5['gencost'][:, 5]  # c0

# scale up some line thermal limits
line_id = np.array([343,  344, 2324])
branch = mpc5['branch'].copy()
branch[:, 5] = 1.1 * mpc5['branch'][:, 5]

mpc_out = mpc5.copy()
mpc_out['gencost'] = gencost
mpc_out['branch'] = branch

In [7]:
sp = ams.system.System()
ams.io.matpower.mpc2system(mpc_out, sp)

sp.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c, amax, amin.
System set up in 0.0077 seconds.


True

In [8]:
sp.DCOPF.run(solver='PIQP')

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.5259 seconds.
<DCOPF> solved as optimal in 0.2016 seconds, converged in 14 iterations with PIQP.


True

Export to XLSX format.

In [9]:
ams.io.xlsx.write(sp, './Texas2k/Texas2k_opf.xlsx', overwrite=True)

xlsx file written to "./Texas2k/Texas2k_opf.xlsx"


True